In [1]:
#%pip install newspaper3k

  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13542 sha256=b3fb49439bfff8cbacf520a54768ef832b6bd2b6026da5f30e42f7e3a5379d14
  Stored in directory: C:\Users\HP\AppData\Local\pip\Cache\wheels\81\2b\43\a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedparser: filename=feedparser-5.2.1-cp37-none-any.whl size=44944 sha256=d751b5ffc4ee854a9ec4e3c0fe88fe3146c4b97e31b7718531f84e54ed8d2cda
  Stored in directory: C:\Users\HP\AppData\Local\pip\Cache\wheels\8c\69\b7\f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3362 sha256=fb6374b2236c5bed842a7464576d506888f02133e06835f9713006c29f9fd2da
  Stored in directory: C:\Users\HP\AppData\Local\pip\Cache\wheels\de\03\ca\778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398414 sha256=41f1aaf583a622455cb347c204689bb4d9d576040caa7ad3a

In [1]:
import nltk
import newspaper
from newspaper import Article
import pandas as pd
import numpy as np
import datetime, time

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
# read links data

df = pd.read_csv("links.csv", delimiter='|', encoding="utf-8", dtype=str)

len(df)

2570

In [36]:
startTime = time.time()

for index, row in df.iterrows():
    try:
        a = Article(row.URL, language='fr') 
        a.download()
        a.parse()
        a.nlp()
    except:
        print('Error scraping')
        pass
    df.at[index, 'domain'] = a.source_url
    df.at[index, 'summary'] = a.summary
    #.replace("\n", " ")
    df.at[index, 'content'] = a.text
    df.at[index, 'title'] = a.title
    df.at[index, 'top_image'] = a.top_image
    df.at[index, 'authors'] = ', '.join([t for t in a.authors])
    df.at[index, 'keywords'] = ', '.join([t for t in a.keywords])
    df.at[index, 'meta_keywords'] = a.meta_keywords
    df.at[index, 'meta_description'] = a.meta_description
    df.at[index, 'tags'] = ', '.join([t for t in a.tags])
    df.at[index, 'published_at'] = a.publish_date.strftime("%d-%m-%Y") if a.publish_date else ''
    df.at[index, 'scraped_at'] = datetime.datetime.utcnow().strftime("%d-%m-%Y")
    #wait 2 seconds
    #time.sleep(1)    
    
#save data to file
#df_test.to_csv('news_data.csv', sep=',', encoding="utf-8")

#show time spent for processing
"{0} seconds".format(round(time.time() - startTime,2))

Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping
Error scraping


'3964.24 seconds'

In [45]:
df.isnull().sum()

URL                 0
id                  0
domain              0
type                0
summary             0
content             0
title               0
top_image           0
authors             0
keywords            0
meta_keywords       0
meta_description    0
tags                0
published_at        0
scraped_at          0
dtype: int64

In [14]:
a = Article("https://secretnews.fr/camping-4/", language='fr') 
a.download()
a.parse()


In [15]:
a.text

''

In [48]:
"helo world"[0:255] + '...'

'helo world...'

#### Get content for https://secretnews.fr (not works with newscraper)

In [49]:
import requests
from bs4 import BeautifulSoup

In [70]:
for index, row in df.loc[(df['domain'] == 'https://secretnews.fr') & (df['content'] == "")].iterrows():
    try:
        html_file = requests.get(df.at[index, 'URL'])
        soup = BeautifulSoup(html_file.text)
        title = soup.select(".entry-title")[0].get_text(separator=" ", strip=True)
        post = soup.select(".entry-content")[0].get_text(separator=" ", strip=True)
        df.at[index, 'title'] = title
        df.at[index, 'content'] = post
        df.at[index, 'summary'] = post[0:255] + '...'
    except:
        print('Error scraping')
        pass


Error scraping
Error scraping
Error scraping


In [73]:
for index, row in df.loc[df['domain'] == 'https://secretnews.fr']:            
    df.at[index, 'content'] = df.at[index, 'content'][50:]
    df.at[index, 'summary'] = df.at[index, 'content'][0:255] + '...'

ValueError: too many values to unpack (expected 2)

In [71]:
df.to_csv('news_data_part_2.csv', sep=',', encoding="utf-8")